In [1]:
from datavideo import VideoLayout
from torch.utils.data import DataLoader

/workspace/joonsm/anaconda3/envs/CLDM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib.pyplot as plt
import torch
import matplotlib.patches as patches

In [3]:
vid = VideoLayout(type='train')

In [4]:
loader = DataLoader(vid, batch_size=2)

In [5]:
src = next(iter(loader))

In [10]:
bbox = src['box_cond']

In [11]:
bbox

tensor([[[ 0.4750,  0.2367, -0.8025, -0.7300],
         [ 0.4675,  0.2367, -0.7975, -0.7300],
         [ 0.4550,  0.2467, -0.7850, -0.7200],
         [ 0.4350,  0.2633, -0.7700, -0.7033],
         [ 0.4050,  0.2833, -0.7525, -0.6767],
         [ 0.3725,  0.3100, -0.7325, -0.6467],
         [ 0.3325,  0.3400, -0.7025, -0.6133],
         [ 0.2825,  0.3767, -0.6650, -0.5700],
         [ 0.2175,  0.4267, -0.6150, -0.5133],
         [ 0.1250,  0.4733, -0.5275, -0.4667],
         [ 0.0050,  0.4867, -0.4525, -0.4867],
         [-0.1500,  0.4600, -0.4275, -0.5833],
         [-0.2475,  0.4433, -0.4675, -0.5700],
         [-0.3225,  0.4200, -0.5350, -0.5567],
         [-0.3925,  0.3833, -0.6125, -0.5800],
         [-0.4550,  0.3433, -0.6750, -0.6167]],

        [[-0.5100,  0.3067, -0.7175, -0.6567],
         [-0.5600,  0.2733, -0.7475, -0.6933],
         [-0.6025,  0.2433, -0.7700, -0.7233],
         [-0.6425,  0.2200, -0.7900, -0.7500],
         [-0.6775,  0.1967, -0.8050, -0.7733],
         [-

In [28]:
index = 0

In [7]:
import torch
import torch.nn as nn

In [8]:
from loss import bbox_pair_tv_loss

In [19]:
from einops import rearrange

In [12]:
bbox.size()

torch.Size([2, 16, 4])

In [20]:
bbb = rearrange(bbox, 'b c d -> (b c) d')

In [15]:
bbox

torch.Size([2, 16, 4])

In [22]:
bbb.size()

torch.Size([32, 4])

In [22]:
bbb.size()

torch.Size([32, 4])

In [20]:
bbox_pair_tv_loss(bbb)

tensor(0.1115)

In [ ]:
bbb.shape[0]//16

2